In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import L2
from xgboost import XGBClassifier

In [3]:
df_train=pd.read_csv("train.csv")
dftest=pd.read_csv("test.csv")

In [4]:
display(df_train.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
columns=['Survived', 'Pclass', 'Sex', 'Age','SibSp','Parch','Fare']
dfmodel=df_train[columns].copy()
display(dfmodel.head(10))

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
5,0,3,male,NaN,0,0,8.4583
6,0,1,male,54.0,0,0,51.8625
7,0,3,male,2.0,3,1,21.0750
8,1,3,female,27.0,0,2,11.1333
9,1,2,female,14.0,1,0,30.0708


In [6]:
dfmodel['sexb']=dfmodel['Sex'].map({'male':1,'female':0})
dfmodel=dfmodel.drop('Sex',axis=1)
display(dfmodel.head(10))

,Survived,Pclass,Age,SibSp,Parch,Fare,sexb
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1
5,0,3,NaN,0,0,8.4583,1
6,0,1,54.0,0,0,51.8625,1
7,0,3,2.0,3,1,21.0750,1
8,1,3,27.0,0,2,11.1333,0
9,1,2,14.0,1,0,30.0708,0


In [7]:
dfmodel['Age']=dfmodel['Age'].fillna(dfmodel['Age'].median())
dfmodel['Age'].isna().sum()
display(dfmodel.head(10))
dfmodel.corr()['Survived']

,Survived,Pclass,Age,SibSp,Parch,Fare,sexb
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1
5,0,3,28.0,0,0,8.4583,1
6,0,1,54.0,0,0,51.8625,1
7,0,3,2.0,3,1,21.0750,1
8,1,3,27.0,0,2,11.1333,0
9,1,2,14.0,1,0,30.0708,0


Survived    1.000000
Pclass     -0.338481
Age        -0.064910
SibSp      -0.035322
Parch       0.081629
Fare        0.257307
sexb       -0.543351
Name: Survived, dtype: float64

In [8]:
y=df_train['Survived']
x=dfmodel.drop('Survived',axis=1)
scaler=StandardScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=40)
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
display(x_train)

array([[ 0.82258588,  0.81624704, -0.47787223, -0.47789902, -0.49985213,
         0.71980808],
       [-1.5876246 , -0.49126054,  0.40818253, -0.47789902,  1.67685731,
        -1.38925921],
       [ 0.82258588, -1.02964601,  3.06634682,  0.76848964,  0.16645084,
         0.71980808],
       ...,
       [ 0.82258588, -2.10641696,  2.18029206,  0.76848964, -0.21557654,
         0.71980808],
       [-0.38251936,  0.04712493, -0.47787223, -0.47789902, -0.43263173,
         0.71980808],
       [ 0.82258588,  2.43140346, -0.47787223, -0.47789902, -0.52012087,
         0.71980808]])

In [9]:
model=XGBClassifier()
model.fit(x_train,y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [22]:
y_pred=model.predict(x_test)

In [23]:
dftest=dftest.sort_values(by='PassengerId')
dftest['Age']=dftest['Age'].fillna(dftest['Age'].median())
dftest['sexb']=dftest['Sex'].map({'male':1,'female':0})
dftest.isna().sum()
dftest['Fare']=dftest['Fare'].fillna(dftest['Fare'].median())
columns=['Pclass', 'Age','SibSp','Parch','Fare', 'sexb']
xpred=dftest[columns]
xpred_scaled=scaler.transform(xpred)
xpred.info()
display(dftest.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Age     418 non-null    float64
 2   SibSp   418 non-null    int64  
 3   Parch   418 non-null    int64  
 4   Fare    418 non-null    float64
 5   sexb    418 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 19.7 KB


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,sexb
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0


In [15]:
acc=accuracy_score(y_test, y_pred)
print(f"Accuracy= {acc:.4f}")
#print(classification_report(y_test,y_pred))=


Accuracy= 0.8324


In [26]:
pred=model.predict(xpred)
binpred=pred
finalsub=pd.DataFrame({
    'PassengerId':dftest['PassengerId'],
    'Survived':binpred.flatten()
})
print(finalsub.head())
finalsub.to_csv("Titanic_v3.csv",index=False)

   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1
